In [1]:
import os,sys,string, time
import ROOT
from math import *
from ROOT import gPad, TTree, TObject, TFile, gDirectory, TH1D, TH2D, TH3D, TCanvas, gROOT, TGaxis, gStyle, TColor, TLegend, THStack, TChain, TLatex, TText, TCollection, kRed, kBlue
from array import array
import matplotlib.pyplot as plt
import numpy as np
import uproot
import pandas as pd
from root_pandas import read_root
from platform import python_version
import scipy
from scipy import stats
import uproot3
import math
from matplotlib.patches import Rectangle
import pickle
import functools
from importlib import reload

import Utilities.Plotter as PT
import Utilities.Constants as Constants
import Utilities.Variables_list as Variables
import Utilities.Functions as Functions
from Utilities.ipython_exit import exit

print ('Success')


Welcome to JupyROOT 6.24/06
Success


# Setting which samples to pickle

In [2]:
Params = {"Run":"run3", #The run number, so far either "run1" or "run3"
          "Load_single_file":False, #This will override everything else, put the desired file in the "single_file" line
          "single_file":'beamgood',
          "Load_lepton_signal":False, #Just e+e- samples
          "Load_standard_bkgs":False, #Backgrounds
          "Load_DetVars":False,
          "Only_keep_common_DetVar_evs":True, #For overlay only atm
          "Load_Signal_DetVars":False,
          "Load_data":False,
          "FLATTEN":True, #Have one row per reconstructed object in the analysis dataframe
          "only_presel":False,#Create small files containing only variables necessary for pre-selection, for making pre-selection plots
          "Load_truth_vars":False, #Load the "mc_" type variables for signal
          "Load_pi0_signal":False,
          "Load_pi0_signal_DetVars":True,
          "Load_sys_vars":False} #Load in the signal samples where the HNL decays to pi0


In [3]:
Params, samples = Functions.create_sample_list(Params)

sample_loc = {"overlay":f'../NuMI_MC/SLIMMED_neutrinoselection_filt_'+Params["Run"]+'_overlay.root',
              "dirtoverlay":f'../NuMI_MC/neutrinoselection_filt_'+Params["Run"]+'_dirt_overlay.root',
              "beamoff":f'../NuMI_data/neutrinoselection_filt_'+Params["Run"]+'_beamoff.root',
              "signal":f'../NuMI_signal/KDAR_dump/sfnues/sfnues_KDAR_dump_',
              "pi0_signal":f'../NuMI_signal/KDAR_dump/sfnues/pi0/sfnues_KDAR_dump_',
              "beamgood":f'../NuMI_data/neutrinoselection_filt_'+Params["Run"]+'_beamon_beamgood.root'}

root_dir = 'nuselection'
main_tree = 'NeutrinoSelectionFilter'
loc_pkls = "pkl_files/"+Params["Run"]+"/current_files/"+Params["variables_string"]+"/"


Loading these run3 samples: 
['150_WireModX', '150_WireModYZ', '150_WireModThetaXZ', '150_WireModThetaYZ', '150_WireModdEdX', '150_LYDown', '150_LYRayleigh', '150_LYAttenuation', '150_SCE', '150_Recomb2', '150_CV', '180_WireModX', '180_WireModYZ', '180_WireModThetaXZ', '180_WireModThetaYZ', '180_WireModdEdX', '180_LYDown', '180_LYRayleigh', '180_LYAttenuation', '180_SCE', '180_Recomb2', '180_CV', '200_WireModX', '200_WireModYZ', '200_WireModThetaXZ', '200_WireModThetaYZ', '200_WireModdEdX', '200_LYDown', '200_LYRayleigh', '200_LYAttenuation', '200_SCE', '200_Recomb2', '200_CV', '220_WireModX', '220_WireModYZ', '220_WireModThetaXZ', '220_WireModThetaYZ', '220_WireModdEdX', '220_LYDown', '220_LYRayleigh', '220_LYAttenuation', '220_SCE', '220_Recomb2', '220_CV', '240_WireModX', '240_WireModYZ', '240_WireModThetaXZ', '240_WireModThetaYZ', '240_WireModdEdX', '240_LYDown', '240_LYRayleigh', '240_LYAttenuation', '240_SCE', '240_Recomb2', '240_CV', '245_WireModX', '245_WireModYZ', '245_WireMod

In [4]:
print(Params["variables_MC"])

['run', 'sub', 'evt', 'nslice', 'n_pfps', 'n_tracks', 'n_showers', 'swtrig_pre', 'swtrig_post', 'trk_sce_start_x_v', 'trk_sce_start_y_v', 'trk_sce_start_z_v', 'trk_sce_end_x_v', 'trk_sce_end_y_v', 'trk_sce_end_z_v', 'shr_theta_v', 'shr_phi_v', 'shr_px_v', 'shr_py_v', 'shr_pz_v', 'shrclusdir0', 'shrclusdir1', 'shrclusdir2', 'shr_energy_tot', 'trk_theta_v', 'trk_phi_v', 'trk_dir_x_v', 'trk_dir_y_v', 'trk_dir_z_v', 'trk_energy', 'trk_energy_hits_tot', 'trk_energy_tot', 'trk_score_v', 'trk_calo_energy_u_v', 'trk_end_x_v', 'trk_chipr_best', 'pfnplanehits_U', 'pfnplanehits_V', 'pfnplanehits_Y', 'NeutrinoEnergy2', 'SliceCaloEnergy2', 'nu_flashmatch_score', 'contained_sps_ratio', 'flash_time', 'contained_fraction', 'trk_score', 'crtveto', 'weightSplineTimesTune', 'ppfx_cv', 'npi0']


In [5]:
if Params["Load_sys_vars"] == True:
    Params["variables_MC"] += Variables.sys_vars
    
print(Params["variables_MC"])

['run', 'sub', 'evt', 'nslice', 'n_pfps', 'n_tracks', 'n_showers', 'swtrig_pre', 'swtrig_post', 'trk_sce_start_x_v', 'trk_sce_start_y_v', 'trk_sce_start_z_v', 'trk_sce_end_x_v', 'trk_sce_end_y_v', 'trk_sce_end_z_v', 'shr_theta_v', 'shr_phi_v', 'shr_px_v', 'shr_py_v', 'shr_pz_v', 'shrclusdir0', 'shrclusdir1', 'shrclusdir2', 'shr_energy_tot', 'trk_theta_v', 'trk_phi_v', 'trk_dir_x_v', 'trk_dir_y_v', 'trk_dir_z_v', 'trk_energy', 'trk_energy_hits_tot', 'trk_energy_tot', 'trk_score_v', 'trk_calo_energy_u_v', 'trk_end_x_v', 'trk_chipr_best', 'pfnplanehits_U', 'pfnplanehits_V', 'pfnplanehits_Y', 'NeutrinoEnergy2', 'SliceCaloEnergy2', 'nu_flashmatch_score', 'contained_sps_ratio', 'flash_time', 'contained_fraction', 'trk_score', 'crtveto', 'weightSplineTimesTune', 'ppfx_cv', 'npi0']


In [6]:
if (Params["Load_DetVars"] == True) and (Params["Only_keep_common_DetVar_evs"] == True):
    df_rse_list = []
    print("Loading only run, subrun, event numbers" + "\n")
    for sample in samples: #Looping over all samples, should make a function for this.
        if sample in Constants.Detector_variations: #Checks if it is an overlay Detector Variation sample
            print(f"Loading overlay {sample} "+Params["Run"]+" with uproot")
            NuMI_MC_overlay = uproot3.open("../NuMI_MC/DetVars/neutrinoselection_filt_"+Params["Run"]+f"_overlay_{sample}.root")[root_dir+"/"+main_tree]
            df_overlay = NuMI_MC_overlay.pandas.df(['run','sub','evt'], flatten=Params["FLATTEN"])
            file = df_overlay
            Functions.make_unique_ev_id(file) #This creates "rse_id" branch
            new_overlay = file.copy()
            df_rse_list.append(new_overlay)

    common_evs = Functions.make_common_evs_df(df_rse_list)
    
elif (Params["Load_Signal_DetVars"] == True) and (Params["Only_keep_common_DetVar_evs"] == True): #WRONG
    common_evs_dict = {}
    print("Loading only run, subrun, event numbers" + "\n")
    if Params["Run"] == "run1": HNL_samples = [150]
    if Params["Run"] == "run3": HNL_samples = [2, 10, 20, 50, 100]
    for HNL_mass in HNL_samples: #Currently don't have 150MeV detvar samples for run3
        df_rse_list = []
        for DetVar in Constants.Detector_variations: #Looping over all samples, should make a function for this.
            #print(f"Loading overlay {sample} "+Params["Run"]+" with uproot")
            NuMI_MC_signal = uproot3.open("../NuMI_signal/KDAR_dump/sfnues/DetVars/"+f"{HNL_mass}_{DetVar}_"+Params["Run"]+".root")[root_dir+"/"+main_tree]
            df_signal = NuMI_MC_signal.pandas.df(['run','sub','evt'], flatten=Params["FLATTEN"])
            file = df_signal
            Functions.make_unique_ev_id(file) #This creates "rse_id" branch
            new_signal = file.copy()
            df_rse_list.append(new_signal)

        common_evs = Functions.make_common_evs_df(df_rse_list)
        common_evs_dict[HNL_mass] = common_evs
        
elif (Params["Load_pi0_signal_DetVars"] == True) and (Params["Only_keep_common_DetVar_evs"] == True): #WRONG
    common_evs_dict = {}
    print("Loading only run, subrun, event numbers" + "\n")
    if Params["Run"] == "run1": HNL_samples = []
    if Params["Run"] == "run3": HNL_samples = [150,180,200,220,240,245]
    for HNL_mass in HNL_samples: #Currently don't have 150MeV detvar samples for run3
        print(HNL_mass)
        df_rse_list = []
        for DetVar in Constants.Detector_variations: #Looping over all samples, should make a function for this.
            #print(f"Loading overlay {sample} "+Params["Run"]+" with uproot")
            NuMI_MC_signal = uproot3.open("../NuMI_signal/KDAR_dump/sfnues/pi0/DetVars/"+f"{HNL_mass}_{DetVar}_"+Params["Run"]+".root")[root_dir+"/"+main_tree]
            df_signal = NuMI_MC_signal.pandas.df(['run','sub','evt'], flatten=Params["FLATTEN"])
            file = df_signal
            Functions.make_unique_ev_id(file) #This creates "rse_id" branch
            new_signal = file.copy()
            df_rse_list.append(new_signal)

        common_evs = Functions.make_common_evs_df(df_rse_list)
        common_evs_dict[HNL_mass] = common_evs
else: common_evs = None

Loading only run, subrun, event numbers

150


/home/david/HNL/ee_decays/Utilities/Functions.py:180: FutureWarning: Passing 'suffixes' which cause duplicate columns {'run_x', 'evt_x', 'sub_x'} in the result is deprecated and will raise a MergeError in a future version.
  overlapping_df = functools.reduce(lambda left,right: pd.merge(left, right, on=['rse_id'], how='inner'), df_list)


Length is of common events list is 9159
180
Length is of common events list is 8672
200
Length is of common events list is 0
220
Length is of common events list is 8732
240
Length is of common events list is 9323
245
Length is of common events list is 9010


## Making new variables

In [7]:
def Make_first_new_variables(df): #Only for variables which can be directly made from the ntuple variables with no manipulation/selection
    df['subcluster'] = df['shrsubclusters0'] + df['shrsubclusters1'] + df['shrsubclusters2']
    df['trkfit'] = df['shr_tkfit_npointsvalid'] / df['shr_tkfit_npoints']
    #Something like shr_tkfit_distance
    return df.copy

#Works, but takes a while, ignored objects with huge negative numbers for the positions.
def Make_fiducial_vars(df):
    print(len(df))
    n_pfps = df["n_pfps"].groupby(level="entry").apply(max)
    df_placeholder = df.query("trk_sce_start_x_v>-1e4").copy()
    print(len(df_placeholder))
    min_x=df_placeholder[["trk_sce_start_x_v","trk_sce_end_x_v"]].min(axis=1).groupby(level="entry").apply(min)
    max_x=df_placeholder[["trk_sce_start_x_v","trk_sce_end_x_v"]].max(axis=1).groupby(level="entry").apply(max)
    
    min_y=df_placeholder[["trk_sce_start_y_v","trk_sce_end_y_v"]].min(axis=1).groupby(level="entry").apply(min)
    max_y=df_placeholder[["trk_sce_start_y_v","trk_sce_end_y_v"]].max(axis=1).groupby(level="entry").apply(max)
    
    min_z=df_placeholder[["trk_sce_start_z_v","trk_sce_end_z_v"]].min(axis=1).groupby(level="entry").apply(min)
    max_z=df_placeholder[["trk_sce_start_z_v","trk_sce_end_z_v"]].max(axis=1).groupby(level="entry").apply(max)
    
    del df_placeholder
    
    df2 = df.copy()
    
    df2["min_x"]=np.array(np.repeat(min_x, np.array(n_pfps)))
    df2["max_x"]=np.array(np.repeat(max_x, np.array(n_pfps)))
    df2["min_y"]=np.array(np.repeat(min_y, np.array(n_pfps)))
    df2["max_y"]=np.array(np.repeat(max_y, np.array(n_pfps)))
    df2["min_z"]=np.array(np.repeat(min_z, np.array(n_pfps)))
    df2["max_z"]=np.array(np.repeat(max_z, np.array(n_pfps)))
    return df2

# Reading in and pickling files

In [9]:
if ((Params["Load_Signal_DetVars"] == True)or(Params["Load_pi0_signal_DetVars"] == True)) and (Params["Only_keep_common_DetVar_evs"] == True):
    Functions.Load_and_pkl_samples(samples, sample_loc, loc_pkls, common_evs_dict, Params, save_str="_FINAL")
else:
    Functions.Load_and_pkl_samples(samples, sample_loc, loc_pkls, common_evs, Params, save_str="_FINAL") #Remove "_New_gen" str for data
print("Done!")


Done!


# Finished code